<a href="https://colab.research.google.com/github/dardan-gashi01/IN3060-CW/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cpu


In [ ]:
import os

# TODO: Fill in the Google Drive path where you uploaded the lab materials
# Example: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Lab materials 01-20210104'

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/CW_Folder_UG/CW_Folder_UG' 
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['test_functions.ipynb', '.DS_Store', 'CW_Dataset.zip', 'Code', 'Models', 'Personal_Dataset']


In [ ]:
zip_path = os.path.join(GOOGLE_DRIVE_PATH, 'CW_Dataset.zip')

!cp '{zip_path}' .

!yes|unzip -q CW_Dataset.zip

!rm CW_Dataset.zip

replace test/test_0288_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_0828_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_1505_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_0317_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_1055_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_0647_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_2986_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_1923_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_0591_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_2326_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_1383_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_2676_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_3064_aligned.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace test/test_2819_aligned.jpg? [y

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
from torch.utils.data import Dataset, DataLoader

%matplotlib inline

In [ ]:
train_dir = 'train'
data_means = [0.485, 0.456, 0.406]
data_stds = [0.229, 0.224, 0.225]

In [53]:
import os, shutil

def dataFolder(trainImgPath, testImgPath, trainLabelPath, testLabelPath):
  #creating empty folders
  os.mkdir('./DataFolder2')
  os.mkdir('./DataFolder2/train')
  os.mkdir('./DataFolder2/test')
  for i in range(7):
    os.mkdir('./DataFolder2/train/' + str(i+1))
    os.mkdir('./DataFolder2/test/' + str(i+1))

  trainPath = './DataFolder2/train'
  testPath = './DataFolder2/test'

  file_name = [file for file in sorted(os.listdir(os.path.join(trainImgPath))) if file.endswith('.jpg')]
  with open ((os.path.join(trainLabelPath)), "r") as myfile:
    data = myfile.read().splitlines()
    for i in data:
      x,y = i.split(" ")
      for file in file_name:
        print("image file: " + file + ' class: ' + y)
  #load images into the correct folders


def make_dir_if_not_exists(path):
  if not os.path.isdir(path):
    os.mkdir(path)


def dataFolder2(trainImgPath, testImgPath, trainLabelPath, testLabelPath):
  #creating empty folders
  make_dir_if_not_exists('./DataFolder2')
  make_dir_if_not_exists('./DataFolder2/train')
  make_dir_if_not_exists('./DataFolder2/test')
  for i in range(1,8):
    make_dir_if_not_exists(f'./DataFolder2/train/{str(i)}')
    make_dir_if_not_exists(f'./DataFolder2/test/{str(i)}')

  trainPath = './DataFolder2/train'
  testPath = './DataFolder2/test'

  file_names = [file for file in os.listdir(trainImgPath) if file.endswith('.jpg')]
  parsed_data = {}
  with open (trainLabelPath, "r") as myfile:
    data = myfile.readlines()
    for line_data in data:
      file_name, number = line_data.strip().split(" ")
      parsed_data[file_name] = number

  for file in file_names:
    shutil.copy(f"{trainImgPath}/{file}", f"{trainPath}/{parsed_data[file.replace('_aligned', '')]}/{file}")
    #print("image file: " + file + ' class: ' + parsed_data[file.replace("_aligned", "")])
  #load images into the correct folders


  file_names = [file for file in os.listdir(testImgPath) if file.endswith('.jpg')]
  parsed_data = {}
  with open (testLabelPath, "r") as myfile:
    data = myfile.readlines()
    for line_data in data:
      file_name, number = line_data.strip().split(" ")
      parsed_data[file_name] = number

  for file in file_names:
    shutil.copy(f"{testImgPath}/{file}", f"{testPath}/{parsed_data[file.replace('_aligned', '')]}/{file}")
    #print("image file: " + file + ' class: ' + parsed_data[file.replace("_aligned", "")])
  #load images into the correct folders

  



In [54]:
dataFolder2('train', 'test', 'labels/list_label_train.txt', 'labels/list_label_test.txt')

['train_00001.jpg 5\n', 'train_00002.jpg 5\n', 'train_00003.jpg 4\n', 'train_00004.jpg 4\n', 'train_00005.jpg 5\n', 'train_00006.jpg 1\n', 'train_00007.jpg 5\n', 'train_00008.jpg 4\n', 'train_00009.jpg 4\n', 'train_00010.jpg 1\n', 'train_00011.jpg 4\n', 'train_00012.jpg 1\n', 'train_00013.jpg 1\n', 'train_00014.jpg 4\n', 'train_00015.jpg 5\n', 'train_00016.jpg 4\n', 'train_00017.jpg 4\n', 'train_00018.jpg 4\n', 'train_00019.jpg 4\n', 'train_00020.jpg 4\n', 'train_00021.jpg 4\n', 'train_00022.jpg 4\n', 'train_00023.jpg 6\n', 'train_00024.jpg 5\n', 'train_00025.jpg 5\n', 'train_00026.jpg 5\n', 'train_00027.jpg 2\n', 'train_00028.jpg 5\n', 'train_00029.jpg 4\n', 'train_00030.jpg 5\n', 'train_00031.jpg 3\n', 'train_00032.jpg 1\n', 'train_00033.jpg 5\n', 'train_00034.jpg 1\n', 'train_00035.jpg 4\n', 'train_00036.jpg 4\n', 'train_00037.jpg 4\n', 'train_00038.jpg 5\n', 'train_00039.jpg 5\n', 'train_00040.jpg 5\n', 'train_00041.jpg 5\n', 'train_00042.jpg 5\n', 'train_00043.jpg 1\n', 'train_000

In [ ]:
file_name = [file for file in sorted(os.listdir(os.path.join('train'))) if file.endswith('.jpg')]
with open ((os.path.join('labels/list_label_train.txt')), "r") as myfile:
  data = myfile.read().splitlines()
  for i in data:
    x,y = i.split(" ")
    for file in file_name:
      print("image file: " + file + ' class: ' + y)

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(data_means, data_stds)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(data_means, data_stds)
    ]),
}

In [ ]:
class faceDataset(Dataset):

  def __init__(self):
    #data loading
    images = []
    file_name = [file for file in sorted(os.listdir(os.path.join('train'))) if file.endswith('.jpg')]
    for file in file_name:
      images.append(plt.imread(os.path.join('train', file)))
    images_array = np.vstack(images)

    labels = []
    with open ((os.path.join('labels/list_label_train.txt')), "r") as myfile:
      data = myfile.read().splitlines()
      for i in data:
        x,z = i.split(" ")
        labels.append(z)
    

    self.X = torch.from_numpy(images_array)
    self.y = torch.from_numpy(np.asarray(labels))
    self.n_samples = np.shape(images_array[0])

  def __getitem__(self, index):
    #dataset [0]
    return self.X[index], self.y[index]

  def __len__(self):
    # len(dataset)
    return self.n_samples



In [ ]:
dataset = faceDataset()

TypeError: ignored